<a href="https://colab.research.google.com/github/solomontessema/building-ai-agents/blob/main/notebooks/5.5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Hybrid and Multihop RAG Pattern</h1></td>
  </tr>
</table>

In [ ]:
!pip install -qU langchain==1.1.0 langchain-openai==1.1.0 langchain-community==0.4.1 faiss-cpu==1.13.2 python-dotenv==1.1.1

In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from dotenv import load_dotenv
import requests

load_dotenv()

# Download file from GitHub
GITHUB_RAW_URL = (
    "https://raw.githubusercontent.com/"
    "solomontessema/building-ai-agents/main/assets/langgraph_intro.txt"
)

response = requests.get(GITHUB_RAW_URL)
response.raise_for_status()

# Create the Index (The "Save" Step)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
docs = [Document(page_content=response.text)]
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("faiss_index")
print("Vector store saved successfully!")

# Load the Index (The "Load" Step)
vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)


In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from dotenv import load_dotenv

load_dotenv()

# Load vector store and retriever
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

# Sub-question decomposition function
def extract_list(text):
    return [line.strip("- ") for line in text.split("\n") if line.startswith("-")]

# Multihop retrieval
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
def multihop_retrieve(query):
    # Add .content here to get the string
    response = llm.invoke(f"Break down into sub-questions:\n{query}")
    steps = response.content

    sub_qs = extract_list(steps)
    docs = []
    for q in sub_qs:
        docs.extend(retriever.invoke(q))
    return docs

# Prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Use the following context to answer the question:
    {context}

    Question:
    {question}
    """
)

# Pipeline
chain = (
    RunnableLambda(lambda q: {
        "context": multihop_retrieve(q),
        "question": q
    })
    | prompt
    | llm
    | StrOutputParser()
)

# Test query
response = chain.invoke("Compare LangGraph and LangChain for workflow control.")
print(response)
